In [6]:
import fairseq
from os.path import join
import torch
from fairseq.models.transformer import TransformerModel

from generator import initialize_graph,get_name
from generator_utils import sparql_decode

In [2]:
fairseq_folder = join('..','..','data','eiopa','5_model_input','fairseq-data-bin-31181')
model_path = join('..','..','models','transformer_iwslt_de_en_20226912')
out_path=model_path + '/out_31181'
graph_path = join('..','..','data','eiopa','1_external')

In [3]:
graph = initialize_graph(graph_path)

Graph has 376260 statements.


In [4]:
model = TransformerModel.from_pretrained(
  '../../models/',
  checkpoint_file='./transformer_iwslt_de_en_20226912/checkpoint_best.pt',
  data_name_or_path='/Users/jan_marcglowienke/Documents/University/Master_Courses/Thesis/3_Code/data/eiopa/5_model_input/fairseq-data-bin-31181',
    tokenizer = 'moses',
   # bpe='subword_nmt',
   # bpe_codes='bpecodes'
)

In [7]:
output = model.translate("In what country does w1963 operate?")
sparql_decode(output)

'SELECT DISTINCT ?a WHERE{?x eiopa-Base:hasEUCountryWhereEntityOperates ?a . ?x eiopa-Base:hasInsuranceUndertakingID w1963 . }'

In [8]:
q = "SELECT DISTINCT ?a WHERE{" \
    "?x eiopa-Base:hasEUCountryWhereEntityOperates ?a ." \
    " ?x eiopa-Base:hasInsuranceUndertakingID 'W1963' . }"

results = graph.query(q)

for line in results:
    for item in line:
        print(get_name(item))

NL


In [62]:
results = []
file = out_path + '/generate-test_1.txt'
out = out_path + '/true_generated_queries_1.txt'
with open(file,'r') as f:
    for line in f.readlines():
        if line.startswith('T') is True:
            head, sentence = line.split('\t')
            # only take index number and sentence, delete \n from end of sentence
            tmp = [int(head.split('-')[1]),sparql_decode(sentence[:-1])]
        elif line.startswith('H') is True:
            head, score, sentence = line.split('\t')
            if int(head.split('-')[1]) == tmp[0]:
                tmp.append(sparql_decode(line.split('\t')[2][:-1]))
                results.append(tmp)
                del tmp
            else: 
                raise ValueError("Mismatch in available true and translated sentences! \n Please check that for every true sentence (T) there is an translation available (H) in the generated file! Error ocurred for translation {}".format(head))
        elif line.startswith('Generate') is True:
            result = line[:-1]
f.close()
with open(out,'w',encoding='utf-8') as target:
    for line in results:
        for item in line:
            target.writelines(str(item) + ', ')
        target.writelines('\n')

In [ ]:
# with open(inp) as f:
#     results = list()
#     for line in filter(lambda x: x.startswith('H'), f.readlines()):
#         line = line.strip()
#         head, score, sentence = line.split('\t')
#         results.append((int(head.split('-')[1]), sentence))
#     results.sort(key=lambda x: x[0])
#     print(*[r[1] for r in results], sep='\n')

In [48]:
result

'Generate test with beam=5: BLEU4 = 85.61, 91.7/88.1/83.3/79.9 (BP=1.000, ratio=1.043, syslen=13932, reflen=13360)'